In [1]:

%env WANDB_NOTEBOOK_NAME Train.ipynb
%env CUDA_DEVICE_ORDER PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES 1

env: WANDB_NOTEBOOK_NAME=Train.ipynb
env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
import pickle
import logging
import gzip
from data import read_all_MP_csv
from tokenization import get_tokens, MASK_SITE

MP_20_cache_data = "cache/mp_20/data.pkl.gz"
MP_20_cache_tensors = "cache/mp_20/tensors.pkl.gz"
MP_20_path = "cdvae/data/mp_20"

try:
    with gzip.open(MP_20_cache_data, "rb") as f:
        datasets_pd, max_len = pickle.load(f)
except Exception as e:
    logging.info("Error reading data cache:")
    logging.info(e)
    logging.info("Reading from csv")
    datasets_pd, max_len = read_all_MP_csv()
    with gzip.open(MP_20_cache_data, "wb") as f:
        pickle.dump((datasets_pd, max_len), f)

In [3]:
try:
    with gzip.open(MP_20_cache_tensors, "rb") as f:
        tensors, site_to_ids, element_to_ids, spacegroup_to_ids = pickle.load(f)
except Exception as e:
    logging.warning("Error reading tensor cache! The token order will change!")
    logging.info(e)
    logging.info("Generating new tensors")
    tensors, site_to_ids, element_to_ids, spacegroup_to_ids = get_tokens(datasets_pd)
    with gzip.open(MP_20_cache_tensors, "wb") as f:
        pickle.dump((tensors, site_to_ids, element_to_ids, spacegroup_to_ids), f)

In [4]:
import torch
device = 'cuda'
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def to_combined_dataset(dataset):
    return dict(
        symmetry_sites=torch.stack(dataset['symmetry_sites_tensor'].to_list()).T.to(device),
        symmetry_elements=torch.stack(dataset['symmetry_elements_tensor'].to_list()).T.to(device),
        spacegroup_number=torch.stack(dataset['spacegroup_number_tensor'].to_list()).to(device),
        padding_mask = torch.stack(dataset['padding_mask_tensor'].to_list()).to(device)
    )
torch_datasets = dict(zip(tensors.keys(), map(to_combined_dataset, tensors.values())))

In [5]:
from wyckoff_transformer import WyckoffTransformerModel, WyckoffTrainer
d_hid = 200  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 4  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 4  # number of heads in ``nn.MultiheadAttention``
dropout = 0.2  # dropout probability
n_space_groups = len(spacegroup_to_ids)
# Nope, not all SGs are present
# assert n_space_groups == 230
model = WyckoffTransformerModel(
    n_space_groups = n_space_groups,
    n_sites = len(site_to_ids),
    n_elements = len(element_to_ids),
    d_space_groups = 16,
    d_sites = 64,
    d_species = 64,
    nhead=nhead,
    d_hid=d_hid,
    nlayers=nlayers,
    dropout=dropout).to(device)

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.10/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
trainer = WyckoffTrainer(model, torch_datasets, max_len, torch.tensor(site_to_ids[MASK_SITE]).to(device))
trainer.train(epochs=2000, val_period=10)

wandb: Currently logged in as: kazeev. Use `wandb login --relogin` to force relogin


New best val_loss_epoch 2.860649824142456, saving the model
Epoch 10 val_loss_epoch 2.860649824142456 saved to checkpoints/2024-01-23_17-04-55/best_model_params.pt
New best val_loss_epoch 1.9290355443954468, saving the model
Epoch 30 val_loss_epoch 1.9290355443954468 saved to checkpoints/2024-01-23_17-04-55/best_model_params.pt
New best val_loss_epoch 1.825156807899475, saving the model
Epoch 50 val_loss_epoch 1.825156807899475 saved to checkpoints/2024-01-23_17-04-55/best_model_params.pt
New best val_loss_epoch 1.7547250986099243, saving the model
Epoch 70 val_loss_epoch 1.7547250986099243 saved to checkpoints/2024-01-23_17-04-55/best_model_params.pt
New best val_loss_epoch 1.6778690814971924, saving the model
Epoch 110 val_loss_epoch 1.6778690814971924 saved to checkpoints/2024-01-23_17-04-55/best_model_params.pt
New best val_loss_epoch 1.6232893466949463, saving the model
Epoch 120 val_loss_epoch 1.6232893466949463 saved to checkpoints/2024-01-23_17-04-55/best_model_params.pt
New be